In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2022-08-09 19:25:38,-23.1203,-134.9692,73.49,78,96,17.07
1,1,Busselton,AU,2022-08-09 19:25:38,-33.6500,115.3333,50.59,69,89,11.32
2,2,Ketchikan,US,2022-08-09 19:21:01,55.3422,-131.6461,66.67,52,0,11.50
3,3,Ambilobe,MG,2022-08-09 19:25:38,-13.2000,49.0500,69.22,71,4,4.85
4,4,Ponta Do Sol,PT,2022-08-09 19:25:39,32.6667,-17.1000,74.03,80,18,5.41


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [12]:


# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=[max(temp, 0) for temp in max_temp], 
    dissipating=False, 
    max_intensity=300, 
    point_radius=4)
    
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent clouds
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of percent Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Faanui,PF,2022-08-09 19:25:39,-16.4833,-151.7500,75.22,74,55,6.55
11,11,Moose Factory,CA,2022-08-09 19:25:41,51.2585,-80.6094,77.00,44,0,20.71
13,13,Lorengau,PG,2022-08-09 19:25:42,-2.0226,147.2712,80.46,77,31,6.82
19,19,Puerto Ayora,EC,2022-08-09 19:25:44,-0.7393,-90.3518,75.16,81,82,7.00
23,23,Vanimo,PG,2022-08-09 19:25:45,-2.6741,141.3028,76.14,87,100,4.43
24,24,Chiang Mai,TH,2022-08-09 19:25:45,18.7500,99.0000,77.07,88,75,5.75
29,29,Hithadhoo,MV,2022-08-09 19:25:47,-0.6000,73.0833,81.39,76,78,4.32
32,32,Bandarbeyla,SO,2022-08-09 19:25:48,9.4942,50.8122,77.58,77,73,29.97
36,36,Sarakhs,IR,2022-08-09 19:25:49,36.5449,61.1577,84.65,21,0,11.68
41,41,Albany,US,2022-08-09 19:22:08,42.6001,-73.9662,83.68,65,85,1.01


In [18]:
preferred_cities_df.count()

City_ID       179
City          179
Country       179
Date          179
Lat           179
Lng           179
Max Temp      179
Humidity      179
Cloudiness    179
Wind Speed    179
dtype: int64

In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Faanui,PF,75.22,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
11,Moose Factory,CA,77.00,51.2585,-80.6094,Moosonee Lilypad
13,Lorengau,PG,80.46,-2.0226,147.2712,Lorengau Harbourside Hotel
19,Puerto Ayora,EC,75.16,-0.7393,-90.3518,Finch Bay Galapagos Hotel
23,Vanimo,PG,76.14,-2.6741,141.3028,Vahmoneh Lodge Vanimo
24,Chiang Mai,TH,77.07,18.7500,99.0000,Centara Riverside Hotel Chiangmai
29,Hithadhoo,MV,81.39,-0.6000,73.0833,Scoop Guest House
32,Bandarbeyla,SO,77.58,9.4942,50.8122,JABIR HOTEL
36,Sarakhs,IR,84.65,36.5449,61.1577,مهمان سرای شرکت گاز
41,Albany,US,83.68,42.6001,-73.9662,


In [31]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [33]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [34]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))